## 聚类：将数据结合在一起
**聚类解决的问题**

给定数据集，比如说鸢尾花的数据集。如果我们知道它有三种类别，但是不知道它们各自的标签。

这个时候我们面临着一个聚类的任务：分割一个没有标签的数据集。

### K-means聚类
K-means是最简单的聚类方法

对于鸢尾花的例子

In [1]:
from sklearn import cluster, datasets
iris = datasets.load_iris()
X_iris = iris.data
y_iris = iris.target

k_means = cluster.KMeans(n_clusters=3)
k_means.fit(X_iris) 

print(k_means.labels_[::10])

print(y_iris[::10])

[2 2 2 2 2 1 1 1 1 1 0 0 0 0 0]
[0 0 0 0 0 1 1 1 1 1 2 2 2 2 2]


### 应用实例：向量量化
通常，聚类算法可以被认为是数据压缩的一种方式。

这个问题有时候可以被认为是[向量量化](https://www.wikiwand.com/zh/%E5%90%91%E9%87%8F%E9%87%8F%E5%8C%96)

例如，可以用于色调分离。

In [3]:
import numpy as np
import scipy as sp
try:
   face = sp.face(gray=True)
except AttributeError:
   from scipy import misc
   face = misc.face(gray=True)
X = face.reshape((-1, 1)) # We need an (n_sample, n_feature) array
k_means = cluster.KMeans(n_clusters=5, n_init=1)
k_means.fit(X) 

values = k_means.cluster_centers_.squeeze()
labels = k_means.labels_
face_compressed = np.choose(labels, values)
face_compressed.shape = face.shape

### 凝聚层次聚类（Hierarchical agglomerative clustering）：ward方法
层次聚类方法是一个典型的方法聚类方法，它建立一个有等级的聚类。

一般来说，各种层次凝聚方法可以分为两个类别：
- 凝聚（Agglomerative）方法：自底向上聚类。当聚类的类别很大时候，计算效率远远小于K-means.
- 分裂（Divisive ）方法：自顶向下聚类。很慢，不建议使用。

#### 特征凝聚（Feature agglomeration）
在监督学习那部分中，我们已经学习了在多维稀疏空间中，使用正则化和lasso方法来避免维度灾难。

另外一种方法是**将相似的特征融合在一起**

In [6]:
digits = datasets.load_digits()
images = digits.images
X = np.reshape(images, (len(images), -1))
connectivity = grid_to_graph(*images[0].shape)

agglo = cluster.FeatureAgglomeration(connectivity=connectivity,
                                     n_clusters=32)
agglo.fit(X) 

X_reduced = agglo.transform(X)

X_approx = agglo.inverse_transform(X_reduced)
images_approx = np.reshape(X_approx, images.shape)

## 降维
### 主成分分析：PCA
PCA选择解释信号最大波动的连续组件

例子如下：
x3特征由x1和x2生成，所以可以用PCA降维成二维

In [8]:
# Create a signal with only 2 useful dimensions
x1 = np.random.normal(size=100)
x2 = np.random.normal(size=100)
x3 = x1 + x2
X = np.c_[x1, x2, x3]

from sklearn import decomposition
pca = decomposition.PCA()
pca.fit(X)


print(pca.explained_variance_)  


# As we can see, only the 2 first components are useful
pca.n_components = 2
X_reduced = pca.fit_transform(X)
X_reduced.shape

[  3.44021152e+00   8.13338862e-01   2.28913486e-32]


(100, 2)

### 独立组件分析（Independent Component Analysis:ICA）
ICA选择那些携带了最多独立信息的部件。其能够恢复非高斯的独立信号。

In [9]:
# Generate sample data
time = np.linspace(0, 10, 2000)
s1 = np.sin(2 * time)  # Signal 1 : sinusoidal signal
s2 = np.sign(np.sin(3 * time))  # Signal 2 : square signal
S = np.c_[s1, s2]
S += 0.2 * np.random.normal(size=S.shape)  # Add noise
S /= S.std(axis=0)  # Standardize data
# Mix data
A = np.array([[1, 1], [0.5, 2]])  # Mixing matrix
X = np.dot(S, A.T)  # Generate observations

# Compute ICA
ica = decomposition.FastICA()
S_ = ica.fit_transform(X)  # Get the estimated sources
A_ = ica.mixing_.T
np.allclose(X,  np.dot(S_, A_) + ica.mean_)

True